In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.resnet_v2 import ResNet101V2
from keras.applications.resnet_v2 import ResNet152V2
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam



datagen = ImageDataGenerator(preprocess_input,
                             horizontal_flip=True,
                             width_shift_range=0.2,
                             validation_split=0.2,
                             )


batch_size = 16

training_generator = datagen.flow_from_directory(
    # dir_path,
    target_size=(224,224),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training',
)

validation_generator = datagen.flow_from_directory(
    # dir_path,
    target_size=(224, 224),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation',
)

vgg16_base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
vgg19_base_model = VGG19(include_top=False, weights='imagenet', input_shape=(224,224,3))
resnet50_base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
resnet101_base_model = ResNet101V2(include_top=False, weights='imagenet', input_shape=(224,224,3))
resnet152_base_model = ResNet152V2(include_top=False, weights='imagenet', input_shape=(224,224,3))

models = {
    'VGG16': vgg16_base_model,
    'VGG19': vgg19_base_model,
    'ResNet50': resnet50_base_model,
    'ResNet101': resnet101_base_model,
    'ResNet152': resnet152_base_model,
}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from keras import models, layers
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet_v2 import ResNet101V2, ResNet152V2
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

input_shape = (224, 224, 3)
num_classes = 5

# Create an ImageDataGenerator
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    width_shift_range=0.2,
    validation_split=0.2
)

batch_size = 16

# Create training and validation generators
train_generator = datagen.flow_from_directory(
    # dir_path,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
)

val_generator = datagen.flow_from_directory(
    # dir_path,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)

def create_model(base_model):
    for layer in base_model.layers:
        layer.trainable = False
    
    # model = models.Sequential([
    #     base_model,
    #     layers.GlobalAveragePooling2D(),
    #     layers.Dense(512, activation='relu'),
    #     layers.Dense(num_classes, activation='softmax')
    # ])

    # Classifier
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu')(x)
    pred = layers.Dense(num_classes, activation='softmax')

    model = models.Model(inputs=base_model.input, outputs=pred)
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create instances of the pre-trained models and compile them
vgg16_model = create_model(VGG16(weights='imagenet', include_top=False, input_shape=input_shape))
vgg19_model = create_model(VGG19(weights='imagenet', include_top=False, input_shape=input_shape))
resnet50_model = create_model(ResNet50(weights='imagenet', include_top=False, input_shape=input_shape))
resnet101_model = create_model(ResNet101V2(weights='imagenet', include_top=False, input_shape=input_shape))
resnet152_model = create_model(ResNet152V2(weights='imagenet', include_top=False, input_shape=input_shape))

# Create a dictionary to hold the compiled models
models_dict = {
    'VGG16': vgg16_model,
    'VGG19': vgg19_model,
    'ResNet50': resnet50_model,
    'ResNet101': resnet101_model,
    'ResNet152': resnet152_model,
}

# Train the models
epochs = 10 

for model_name, model in models_dict.items():
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
    )

    # You can use the history object to analyze training/validation metrics or make predictions
    # For example, plotting the training history
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
